In [1]:
import pandas as pd
import io
import json
import os
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload

# ======================================================
# 1. Detectar entorno (Colab vs GitHub / local)
# ======================================================
def get_credentials():
    """
    - En Colab: usa MI_JSON desde userdata
    - En GitHub / local: usa variable de entorno MI_JSON
    """
    try:
        # --- Colab ---
        from google.colab import userdata
        mi_json = userdata.get("MI_JSON")
        if mi_json is None:
            raise ValueError("MI_JSON no encontrado en Colab userdata")
        info = json.loads(mi_json)
        print("Entorno detectado: Google Colab")
    except Exception:
        # --- GitHub / local ---
        mi_json = os.environ.get("MI_JSON")
        if mi_json is None:
            raise ValueError("MI_JSON no encontrado como variable de entorno")
        info = json.loads(mi_json)
        print("Entorno detectado: GitHub / local")

    return Credentials.from_service_account_info(
        info,
        scopes=[
            "https://www.googleapis.com/auth/drive.readonly",
            "https://www.googleapis.com/auth/spreadsheets.readonly",
        ],
    )

creds = get_credentials()

# ======================================================
# 2. Conexión a Google Drive API
# ======================================================
drive_service = build("drive", "v3", credentials=creds)

# ======================================================
# 3. Parámetros del archivo
# ======================================================
FILE_ID = "1NoscZ0koPUkpYk5B6S8J0NLB1AKTyjoU"
SHEET_NAME = "Diciembre 2025"

# ======================================================
# 4. Detectar tipo de archivo
# ======================================================
meta = drive_service.files().get(
    fileId=FILE_ID,
    fields="id,name,mimeType"
).execute()

mime = meta["mimeType"]
print(f"Archivo: {meta['name']}")
print(f"Tipo: {mime}")

# ======================================================
# 5. Descargar / exportar a memoria
# ======================================================
buffer = io.BytesIO()

if mime == "application/vnd.google-apps.spreadsheet":
    # Google Sheets → exportar a Excel
    request = drive_service.files().export_media(
        fileId=FILE_ID,
        mimeType="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
    )
else:
    # Excel u otro binario
    request = drive_service.files().get_media(fileId=FILE_ID)

downloader = MediaIoBaseDownload(buffer, request)
done = False
while not done:
    _, done = downloader.next_chunk()

buffer.seek(0)

# ======================================================
# 6. Leer hoja específica con pandas
# ======================================================
df = pd.read_excel(buffer, sheet_name=SHEET_NAME)

df.head()

Entorno detectado: Google Colab
Archivo: Asignaciones de Cartera Sep25-Dic25.xlsx
Tipo: application/vnd.openxmlformats-officedocument.spreadsheetml.sheet


,Referencia,Cedula,Nombre del cliente,Id deuda,correo,Deudas Activas,Banco,Número de Crédito,Deuda Resuelve,DBT,...,P_Cierre,I_ESPERADO,Negociador,Tipo Elegible,P_Cierre_norm,Días Atraso liquidacion,Estado PL,Ultima Liquidacion,dias_desde_ultima_liq,Priority_level
0,44032,16539456,Harrinson Arley Cuero Tenorio,629642,harry_12tenorio@hotmail.com,1,Banco Popular,6.60E+08,3834100.0,9750700.0,...,0.056375,17832.159469,Steven Mateo Aroca Garzon,NaN,0.281875,758.0,Atrasado,2022-09-16,1176.0,Prioridad 1
1,48378,91490364,Alexander Gomez Duran,658347,alexgomezduran@hotmail.com,2,BBVA,618794133,4137000.0,59682332.0,...,0.091274,39217.510035,Steven Mateo Aroca Garzon,NaN,0.456369,1257.0,Atrasado,2021-11-26,1470.0,Prioridad 1
2,48378,91490364,Alexander Gomez Duran,658348,alexgomezduran@hotmail.com,2,Davivienda,400107477,9716000.0,59682332.0,...,0.042185,51213.643250,Steven Mateo Aroca Garzon,NaN,0.295298,280.0,Atrasado,2021-11-26,1470.0,Prioridad 1
3,49322,17658639,Robinson Rodriguez Triana,664009,rotri_78@hotmail.com,1,Davivienda,´5917186000515781,8939539.0,15523006.0,...,0.040268,50762.026833,Wendy Vanessa Castillo Velasquez,NaN,0.281875,NaN,Al día,2021-10-25,1502.0,Prioridad 1
4,51492,31173279,Janeth Jaramillo Gonzalez,680841,jj_jara_gonzalez@live.com,2,Serfinanza,636853406,3239500.0,44920100.0,...,0.065196,29201.505528,Sharon Natalia Hernandez Dominguez,NaN,0.456369,NaN,Al día,2022-08-16,1207.0,Prioridad 1


In [2]:
import pandas as pd
import json
import os
import gspread
from gspread_dataframe import get_as_dataframe

# ======================================================
# 1) Detectar entorno + cargar MI_JSON (Colab o GitHub/local)
# ======================================================
def cargar_creds_dict():
    # Intentar Colab
    try:
        from google.colab import userdata
        mi_json = userdata.get("MI_JSON")
        if mi_json:
            print("Entorno detectado: Google Colab")
            return json.loads(mi_json)
    except Exception:
        pass

    # GitHub/local
    mi_json = os.environ.get("MI_JSON")
    if not mi_json:
        raise ValueError("No encontré MI_JSON ni en Colab userdata ni como variable de entorno.")
    print("Entorno detectado: GitHub / local")
    return json.loads(mi_json)

creds_dict = cargar_creds_dict()

# ======================================================
# 2) Cliente gspread con Service Account (sirve en ambos entornos)
# ======================================================
gc = gspread.service_account_from_dict(creds_dict)

# ======================================================
# 3) Abrir Google Sheet + seleccionar hoja por gid
# ======================================================
url = "https://docs.google.com/spreadsheets/d/1PogjYq2cMoDpy8gJ5I3-puLvAs0y5788Qv81KSx85Ms/edit#gid=768376152"
gid = 768376152

sh = gc.open_by_url(url)
worksheet = sh.get_worksheet_by_id(gid)

# ======================================================
# 4) Convertir a DataFrame
# ======================================================
df_posibles = get_as_dataframe(worksheet, evaluate_formulas=True)

# Limpieza típica: quitar filas completamente vacías
df_posibles = df_posibles.dropna(how="all")

df_posibles.head()

Entorno detectado: Google Colab


,referencia,Nombre completo,Id deuda,Tipo de Cobro,Vehiculo de ahorro,Comisión Mensual,Apartado mensual,Tipo de liquidación,Potencial,Deudas activas,...,Saldo,Apartado neto disponible,Depósitos para 30%,Apartados necesarios Pago Total,Ahorro / Deuda,Status Credito/Estr,Liquidado,Negociador,Lider,Apartados PaB
0,42093180.0,Claudia Patricia Gonzalez Tobon,1838351.0,Pricing,skandia,837099.0,2950620.0,Tradicional,2.0,7.0,...,-1.115321e+07,2113521.0,1.967225,-5.068396,2.314634,0.0,Sin Liquidar,Alba Yohana Moreno Martin,Natalia Valentina Castro Jimenez,-5.366346
1,42093180.0,Claudia Patricia Gonzalez Tobon,1838350.0,Pricing,skandia,837099.0,2950620.0,Tradicional,0.0,7.0,...,-1.448431e+07,2113521.0,4.301636,5.253851,0.432653,0.0,Sin Liquidar,Alba Yohana Moreno Martin,Natalia Valentina Castro Jimenez,3.893544
2,42093180.0,Claudia Patricia Gonzalez Tobon,1838353.0,Pricing,skandia,837099.0,2950620.0,Tradicional,3.0,7.0,...,-9.434644e+06,2113521.0,3.529210,4.671301,0.425470,0.0,Sin Liquidar,Alba Yohana Moreno Martin,Natalia Valentina Castro Jimenez,3.039742
3,52098105.0,Sandra Patricia Prieto Leiton,1875944.0,Pricing,skandia,322524.0,1206918.0,Tradicional,1.0,4.0,...,-2.448526e+06,884394.0,1.127086,-2.664043,2.307232,0.0,Sin Liquidar,Alba Yohana Moreno Martin,Natalia Valentina Castro Jimenez,-2.851162
4,52098105.0,Sandra Patricia Prieto Leiton,1875945.0,Pricing,skandia,322524.0,1206918.0,Tradicional,3.0,4.0,...,-7.402353e+05,884394.0,2.837893,-0.805390,0.966679,0.0,Sin Liquidar,Alba Yohana Moreno Martin,Natalia Valentina Castro Jimenez,-1.203716


In [3]:
df_posibles['referencia'] = df_posibles['referencia'].astype('Int64').astype(str)
df['Referencia'] = df['Referencia'].astype('Int64').astype(str)

In [4]:
referencias_validas = set(df_posibles['referencia'])

df_filtrado = df[df['Referencia'].isin(referencias_validas)]

In [5]:
df_final = df_filtrado[
    (df_filtrado["Priority_level"] == "Prioridad 1") |
    ((df_filtrado["Priority_level"] == "Prioridad 2") &
     (df_filtrado["Estructurable"] == 1))
]

In [7]:
import json
import gspread
from google.oauth2.service_account import Credentials

# --- Cargar MI_JSON (Colab / GitHub) ---
try:
    from google.colab import userdata
    mi_json = userdata.get("MI_JSON")
except ImportError:
    import os
    mi_json = os.environ.get("MI_JSON")

creds_dict = json.loads(mi_json)
scopes = ["https://www.googleapis.com/auth/spreadsheets.readonly"]
creds = Credentials.from_service_account_info(creds_dict, scopes=scopes)
gc = gspread.authorize(creds)

SPREADSHEET_ID = "1O8OHuVhgwhLw8XYEBf1uBzLYrxQ45rPiZecHOnAa1Go"
GID = 1739799528

sh = gc.open_by_key(SPREADSHEET_ID)
ws = sh.get_worksheet_by_id(GID)

print("✅ Spreadsheet:", sh.title)
print("✅ Worksheet title:", ws.title)
print("✅ Rows:", ws.row_count, "| Cols:", ws.col_count)

✅ Spreadsheet: Actualizaciones
✅ Worksheet title: act 2025
✅ Rows: 280101 | Cols: 7


In [8]:
import json
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials

# =====================================
# 1. Cargar MI_JSON (Colab / GitHub)
# =====================================
try:
    from google.colab import userdata
    mi_json = userdata.get("MI_JSON")
except ImportError:
    import os
    mi_json = os.environ.get("MI_JSON")

if mi_json is None:
    raise ValueError("MI_JSON no encontrado")

creds_dict = json.loads(mi_json)

scopes = ["https://www.googleapis.com/auth/spreadsheets.readonly"]
creds = Credentials.from_service_account_info(creds_dict, scopes=scopes)

gc = gspread.authorize(creds)

# =====================================
# 2. Abrir Sheet
# =====================================
SPREADSHEET_ID = "1O8OHuVhgwhLw8XYEBf1uBzLYrxQ45rPiZecHOnAa1Go"
GID = 1739799528

sh = gc.open_by_key(SPREADSHEET_ID)
ws = sh.get_worksheet_by_id(GID)

df_12098 = pd.DataFrame(ws.get_all_records())

print(df_12098.head())
print(df_12098.shape)

  expiration                                       observations  \
0             Stefy 8086 // 7:22 // Se valida en chat de tuy...   
1             7:22 a. m. 02/01/2025 VivianR//8522// Crédito ...   
2             YesseniaJ|8045|02-01-2025|9:07am|llamo a tt pa...   
3             Actividad para Nicolle Stefy 8086 // 10:52 // ...   
4             7:22 a. m. 02/01/2025 VivianR//8522// Crédito ...   

                               end  debt_id   payment_to_bank  \
0     stefany.rojas@gobravo.com.co  1508614   (551000000,COP)   
1  vivian.rodriguez@gobravo.com.co  1186799   (500000000,COP)   
2      rosa.jimenez@gobravo.com.co  1305562    (54700000,COP)   
3     stefany.rojas@gobravo.com.co   873948  (2600000000,COP)   
4  vivian.rodriguez@gobravo.com.co  1450752   (573800000,COP)   

           inserted_at  bank_reference  
0  2025-01-02 12:23:03        51911010  
1  2025-01-02 12:48:27      3108379218  
2  2025-01-02 14:08:00      3103940951  
3  2025-01-02 15:58:43      3158318257  
4

In [9]:
df_12098

,expiration,observations,end,debt_id,payment_to_bank,inserted_at,bank_reference
0,,Stefy 8086 // 7:22 // Se valida en chat de tuy...,stefany.rojas@gobravo.com.co,1508614,"(551000000,COP)",2025-01-02 12:23:03,51911010
1,,7:22 a. m. 02/01/2025 VivianR//8522// Crédito ...,vivian.rodriguez@gobravo.com.co,1186799,"(500000000,COP)",2025-01-02 12:48:27,3108379218
2,,YesseniaJ|8045|02-01-2025|9:07am|llamo a tt pa...,rosa.jimenez@gobravo.com.co,1305562,"(54700000,COP)",2025-01-02 14:08:00,3103940951
3,,Actividad para Nicolle Stefy 8086 // 10:52 // ...,stefany.rojas@gobravo.com.co,873948,"(2600000000,COP)",2025-01-02 15:58:43,3158318257
4,,7:22 a. m. 02/01/2025 VivianR//8522// Crédito ...,vivian.rodriguez@gobravo.com.co,1450752,"(573800000,COP)",2025-01-02 12:44:39,3022621202
...,...,...,...,...,...,...,...
280095,,1:59 pm || Franchesca || Omni || Se valida con...,franchesca.pereira@gobravo.com.co,1839751,"(643642300,COP)",2025-12-15 19:00:18,1055479612
280096,,Gaby S // 13:58 // Se retorna llamada tt indic...,gabriela.saavedra@gobravo.com.co,1785488,"(321200000,COP)",2025-12-15 19:00:45,3004251571
280097,,2:01 pm |Tatiana Malaver| Se recibe descuento ...,suleimy.malaver@gobravo.com.co,1747022,"(243033100,COP)",2025-12-15 19:01:33,3108647147
280098,,Jana || 2:03 p. m. 15/12/2025 se valida por wh...,jana.lopez@gobravo.com.co,1814804,"(788000000,COP)",2025-12-15 19:04:20,3208205648


In [10]:
df_12098["payment_to_bank_num"] = (
    df_12098["payment_to_bank"]
    .astype(str)
    .str.extract(r"\((\d+),")   # extrae solo el número antes de la coma
    .astype("Int64")            # lo convierte a número entero (permite NaN)
)

In [11]:
# 1. Asegurar que las llaves tengan el mismo tipo
df_posibles["Id deuda"] = df_posibles["Id deuda"].astype("Int64")
df_final["Id deuda"]    = df_final["Id deuda"].astype("Int64")

# 2. Merge SIN alterar la cantidad de filas de df_final
df_final = df_final.merge(
    df_posibles[
        [
            "Id deuda",
            "Apartado neto disponible",
            "Depósitos para 30%",
            "Apartados necesarios Pago Total",
            "Ahorro / Deuda",
            "Comisión total",
        ]
    ],
    how="left",   # mantiene filas de df_final
    on="Id deuda"
)

# 3. Verificar que el número de filas siga siendo el mismo
print("Filas originales:", df_final.shape[0])  # después del merge debe ser igual
df_final.head()

Filas originales: 4934


/tmp/ipython-input-540771249.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["Id deuda"]    = df_final["Id deuda"].astype("Int64")


,Referencia,Cedula,Nombre del cliente,Id deuda,correo,Deudas Activas,Banco,Número de Crédito,Deuda Resuelve,DBT,...,Días Atraso liquidacion,Estado PL,Ultima Liquidacion,dias_desde_ultima_liq,Priority_level,Apartado neto disponible,Depósitos para 30%,Apartados necesarios Pago Total,Ahorro / Deuda,Comisión total
0,5829693,5829693,Rudi Andres lasso Giraldo,1862220,rudyandreslasso@hotmail.es,9,Banco de Bogota,450668827,4841100.0,49700682.0,...,NaN,Al día,NaT,NaN,Prioridad 2,NaN,NaN,NaN,NaN,NaN
1,5829693,5829693,Rudi Andres lasso Giraldo,1862217,rudyandreslasso@hotmail.es,9,Banco de Bogota,459918216,4187700.0,49700682.0,...,856.0,Atrasado,NaT,NaN,Prioridad 1,NaN,NaN,NaN,NaN,NaN
2,5829693,5829693,Rudi Andres lasso Giraldo,1862221,rudyandreslasso@hotmail.es,9,Colsubsidio,093518359,5179900.0,49700682.0,...,1082.0,Atrasado,NaT,NaN,Prioridad 1,NaN,NaN,NaN,NaN,NaN
3,5829693,5829693,Rudi Andres lasso Giraldo,1862216,rudyandreslasso@hotmail.es,9,Banco de Bogota,000099902,3349500.0,49700682.0,...,743.0,Atrasado,NaT,NaN,Prioridad 1,NaN,NaN,NaN,NaN,NaN
4,5829693,5829693,Rudi Andres lasso Giraldo,1862222,rudyandreslasso@hotmail.es,9,Falabella,00201704114130,5402782.0,49700682.0,...,0.0,Al día,NaT,NaN,Prioridad 2,NaN,NaN,NaN,NaN,NaN


In [12]:
# 1. Asegurar que ambos estén como string (evita problemas de 12345 vs 12345.0)
df_final["Referencia"] = df_final["Referencia"].astype("Int64").astype(str)
df_12098["bank_reference"] = df_12098["bank_reference"].astype("Int64").astype(str)

In [13]:
# Convertir a datetime
df_12098["inserted_at"] = pd.to_datetime(df_12098["inserted_at"], errors="coerce")

# Dejar solo la fecha (sin hora)
df_12098["inserted_at"] = df_12098["inserted_at"].dt.date

In [14]:
import pandas as pd

# 1. Asegurar tipos correctos para llaves y fecha
df_12098["debt_id"] = df_12098["debt_id"].astype("Int64")

# Si aún no lo habías hecho:
df_12098["inserted_at"] = pd.to_datetime(df_12098["inserted_at"], errors="coerce")

# Si quieres quedarte solo con la fecha (sin hora):
df_12098["inserted_at"] = df_12098["inserted_at"].dt.date

df_final["Id deuda"] = df_final["Id deuda"].astype("Int64")

# 2. Quedarnos solo con la ÚLTIMA actualización por debt_id
df_12098_last = (
    df_12098
    .sort_values("inserted_at")                          # ordenar por fecha
    .drop_duplicates(subset="debt_id", keep="last")      # conservar la última por deuda
    [["debt_id", "inserted_at", "payment_to_bank_num"]]  # solo columnas necesarias
)

# 3. Renombrar columnas para el merge final
df_12098_last = df_12098_last.rename(
    columns={
        "inserted_at": "ult_act",
        "payment_to_bank_num": "Pab"
    }
)

# 4. Hacer el merge con df_final usando Id deuda
df_final = df_final.merge(
    df_12098_last,
    how="left",
    left_on="Id deuda",
    right_on="debt_id"
)

# 5. Eliminar columna auxiliar debt_id
df_final = df_final.drop(columns=["debt_id"])

# 6. Chequeo rápido
print(df_final[["Id deuda", "ult_act", "Pab"]].head())
print(df_final[["ult_act", "Pab"]].isna().mean())

   Id deuda     ult_act        Pab
0   1862220  2025-12-03  484110000
1   1862217  2025-12-03  418770000
2   1862221         NaN       <NA>
3   1862216  2025-12-03  334950000
4   1862222  2025-10-23  625500000
ult_act    0.604783
Pab        0.604783
dtype: float64


In [15]:
df_final

,Referencia,Cedula,Nombre del cliente,Id deuda,correo,Deudas Activas,Banco,Número de Crédito,Deuda Resuelve,DBT,...,Ultima Liquidacion,dias_desde_ultima_liq,Priority_level,Apartado neto disponible,Depósitos para 30%,Apartados necesarios Pago Total,Ahorro / Deuda,Comisión total,ult_act,Pab
0,5829693,5829693,Rudi Andres lasso Giraldo,1862220,rudyandreslasso@hotmail.es,9,Banco de Bogota,450668827,4841100.0,49700682.0,...,NaT,NaN,Prioridad 2,NaN,NaN,NaN,NaN,NaN,2025-12-03,484110000
1,5829693,5829693,Rudi Andres lasso Giraldo,1862217,rudyandreslasso@hotmail.es,9,Banco de Bogota,459918216,4187700.0,49700682.0,...,NaT,NaN,Prioridad 1,NaN,NaN,NaN,NaN,NaN,2025-12-03,418770000
2,5829693,5829693,Rudi Andres lasso Giraldo,1862221,rudyandreslasso@hotmail.es,9,Colsubsidio,093518359,5179900.0,49700682.0,...,NaT,NaN,Prioridad 1,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
3,5829693,5829693,Rudi Andres lasso Giraldo,1862216,rudyandreslasso@hotmail.es,9,Banco de Bogota,000099902,3349500.0,49700682.0,...,NaT,NaN,Prioridad 1,NaN,NaN,NaN,NaN,NaN,2025-12-03,334950000
4,5829693,5829693,Rudi Andres lasso Giraldo,1862222,rudyandreslasso@hotmail.es,9,Falabella,00201704114130,5402782.0,49700682.0,...,NaT,NaN,Prioridad 2,NaN,NaN,NaN,NaN,NaN,2025-10-23,625500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4929,3508329735,1032480298,KAREN DAYANA RUEDA PENA,2016621,karueda03@hotmail.com,5,Davivienda,459321109,8723000.0,81779500.0,...,NaT,NaN,Prioridad 2,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
4930,3508329735,1032480298,KAREN DAYANA RUEDA PENA,2016620,karueda03@hotmail.com,5,Nu Bank,000001981,7260000.0,81779500.0,...,NaT,NaN,Prioridad 2,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
4931,3508980827,52417868,Norma Katerina Bernier Barliza,1897721,kata_1977@hotmail.com,4,Banco de Bogota,000091552,2807700.0,16361900.0,...,NaT,NaN,Prioridad 1,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
4932,3508980827,52417868,Norma Katerina Bernier Barliza,1897719,kata_1977@hotmail.com,4,Serfinanza,543280844,1458600.0,16361900.0,...,NaT,NaN,Prioridad 1,225355.0,5.963249,6.09483,0.0,42360.58827,NaN,<NA>


In [16]:
columnas_finales = [
    "Referencia",
    "Cedula",
    "Nombre del cliente",
    "Id deuda",
    "correo",
    "Banco",
    "Deuda Resuelve",
    "DBT",
    "tipo_cliente",
    "Comisión Mensual",
    "Ahorro",
    "Tipo de cobro",
    "Tipo de Liquidacion",
    "CE",
    "Fecha PL",
    "BANCOS_ESTANDAR",
    "Descuento",
    "P_cierre_Credito_norm",
    "P_Cierre",
    "I_ESPERADO",
    "Negociador",
    "Tipo Elegible",
    "P_Cierre_norm",
    "Días Atraso liquidacion",
    "Estado PL",
    "Ultima Liquidacion",
    "dias_desde_ultima_liq",
    "Priority_level",
    "Apartado neto disponible",
    "Depósitos para 30%",
    "Apartados necesarios Pago Total",
    "Ahorro / Deuda",
    "Comisión total",
    "ult_act",
    "Pab"
]

# Filtrar el dataframe
df_final = df_final[columnas_finales]

# Ver resultado
df_final.head()
print(df_final.shape)

(4934, 35)


In [17]:
columnas_requeridas = [
    "Apartado neto disponible",
    "Depósitos para 30%",
    "Apartados necesarios Pago Total",
    "Ahorro / Deuda",
    "Comisión total"
]

df_final = df_final.dropna(subset=columnas_requeridas)

In [18]:
bancos_excluir = [
    'Banco Davivienda',
    'Fincomercio',
    'Mundo Mujer',
    'Comultrasan',
    'Juancho te Presta',
    'JOHN',
    'SisteCredito',
    'Zinobe'
]

df_final = df_final[~df_final["BANCOS_ESTANDAR"].isin(bancos_excluir)]

In [19]:
df_final

,Referencia,Cedula,Nombre del cliente,Id deuda,correo,Banco,Deuda Resuelve,DBT,tipo_cliente,Comisión Mensual,...,Ultima Liquidacion,dias_desde_ultima_liq,Priority_level,Apartado neto disponible,Depósitos para 30%,Apartados necesarios Pago Total,Ahorro / Deuda,Comisión total,ult_act,Pab
5,5829693,5829693,Rudi Andres lasso Giraldo,1862215,rudyandreslasso@hotmail.es,Falabella,1104258.0,49700682.0,bravo_co,249076.20,...,NaT,NaN,Prioridad 2,584505.0,3.618589,4.046986,0.366558,49277.51325,2025-10-23,159000000
6,5829693,5829693,Rudi Andres lasso Giraldo,1862219,rudyandreslasso@hotmail.es,Av Villas,4468142.0,49700682.0,bravo_co,249076.20,...,NaT,NaN,Prioridad 2,584505.0,4.914433,1.312854,0.533825,279147.17145,NaN,<NA>
7,5829693,5829693,Rudi Andres lasso Giraldo,1862218,rudyandreslasso@hotmail.es,Scotiabank Colpatria,4361500.0,49700682.0,bravo_co,249076.20,...,NaT,NaN,Prioridad 1,584505.0,2.672379,-0.656360,0.546878,516132.75000,2025-12-15,147000000
9,10932694,10932694,ARMANDO ILICK HUMANEZ USTA,1922148,armandousta@hotmail.com,Banco AV Villas,3383503.0,73687327.0,bravo_co,363251.44,...,NaT,NaN,Prioridad 1,592480.0,3.888485,1.885912,0.418396,249634.85134,NaN,<NA>
12,13715630,13715630,FREDY CASTELLANOS HERRENO,1840944,martilujosfredy@hotmail.com,Banco Finandina,3308800.0,126055900.0,bravo_co,462519.32,...,NaT,NaN,Prioridad 2,1212976.0,0.624522,6.184390,0.357846,225223.39840,2025-09-23,330880000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4920,3507660604,40032509,Maria Consuelo Puerto Hurtado,1935450,mcph35@hotmail.com,Itaú,3752700.0,98043730.0,bravo_co,479187.91,...,NaT,NaN,Prioridad 1,735671.0,4.246981,5.448418,0.198052,480064.14750,NaN,<NA>
4921,3507709067,43281300,Gloria Ines Gomez Restrepo,1956015,gloriagomezrestrepo66@gmail.com,Banco Popular,2328700.0,36413800.0,bravo_co,229163.06,...,NaT,NaN,Prioridad 1,312212.0,5.482110,4.741056,0.134340,116388.42600,NaN,<NA>
4924,3507978734,1090494112,VICTOR MANUEL PATINO HERNANDEZ,1991440,victorpatino1996@gmail.com,Scotiabank Colpatria,652520.0,8967483.0,bravo_co,65857.48,...,NaT,NaN,Prioridad 1,134894.0,3.418024,2.656493,0.196164,36107.19420,NaN,<NA>
4926,3508329735,1032480298,KAREN DAYANA RUEDA PENA,2016618,karueda03@hotmail.com,Falabella,1240800.0,81779500.0,bravo_co,0.00,...,NaT,NaN,Prioridad 2,1027618.0,2.034334,2.114591,0.000172,118124.16000,NaN,<NA>


In [20]:
df

,Referencia,Cedula,Nombre del cliente,Id deuda,correo,Deudas Activas,Banco,Número de Crédito,Deuda Resuelve,DBT,...,P_Cierre,I_ESPERADO,Negociador,Tipo Elegible,P_Cierre_norm,Días Atraso liquidacion,Estado PL,Ultima Liquidacion,dias_desde_ultima_liq,Priority_level
0,44032,16539456,Harrinson Arley Cuero Tenorio,629642,harry_12tenorio@hotmail.com,1,Banco Popular,6.60E+08,3834100.0,9750700.0,...,0.056375,17832.159469,Steven Mateo Aroca Garzon,NaN,0.281875,758.0,Atrasado,2022-09-16,1176.0,Prioridad 1
1,48378,91490364,Alexander Gomez Duran,658347,alexgomezduran@hotmail.com,2,BBVA,618794133,4137000.0,59682332.0,...,0.091274,39217.510035,Steven Mateo Aroca Garzon,NaN,0.456369,1257.0,Atrasado,2021-11-26,1470.0,Prioridad 1
2,48378,91490364,Alexander Gomez Duran,658348,alexgomezduran@hotmail.com,2,Davivienda,400107477,9716000.0,59682332.0,...,0.042185,51213.643250,Steven Mateo Aroca Garzon,NaN,0.295298,280.0,Atrasado,2021-11-26,1470.0,Prioridad 1
3,49322,17658639,Robinson Rodriguez Triana,664009,rotri_78@hotmail.com,1,Davivienda,´5917186000515781,8939539.0,15523006.0,...,0.040268,50762.026833,Wendy Vanessa Castillo Velasquez,NaN,0.281875,NaN,Al día,2021-10-25,1502.0,Prioridad 1
4,51492,31173279,Janeth Jaramillo Gonzalez,680841,jj_jara_gonzalez@live.com,2,Serfinanza,636853406,3239500.0,44920100.0,...,0.065196,29201.505528,Sharon Natalia Hernandez Dominguez,NaN,0.456369,NaN,Al día,2022-08-16,1207.0,Prioridad 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45709,3508859961,1018468661,MAYRA ALEJANDRA HERRERA YEPES,2003472,mayrahe.ye@gmail.com,2,Zinobe,69248468,279500.0,5111720.0,...,0.065579,384.916420,Yithza Camila Paez Lopez,NaN,0.459054,645.0,Atrasado,NaT,NaN,Prioridad 1
45710,3508980827,52417868,Norma Katerina Bernier Barliza,1897722,kata_1977@hotmail.com,4,Bancolombia,990091508,9451200.0,16361900.0,...,0.126534,102249.208861,Dayana Isabel Ojito Ortiz,NaN,0.632669,219.0,Atrasado,NaT,NaN,Prioridad 2
45711,3508980827,52417868,Norma Katerina Bernier Barliza,1897721,kata_1977@hotmail.com,4,Banco de Bogota,000091552,2807700.0,16361900.0,...,0.172227,19250.559083,Dayana Isabel Ojito Ortiz,NaN,0.861133,690.0,Atrasado,NaT,NaN,Prioridad 1
45712,3508980827,52417868,Norma Katerina Bernier Barliza,1897719,kata_1977@hotmail.com,4,Serfinanza,543280844,1458600.0,16361900.0,...,0.133061,6631.248121,Dayana Isabel Ojito Ortiz,NaN,0.931430,528.0,Atrasado,NaT,NaN,Prioridad 1


In [21]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2000 entries, 5 to 4932
Data columns (total 35 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   Referencia                       2000 non-null   object        
 1   Cedula                           2000 non-null   object        
 2   Nombre del cliente               2000 non-null   object        
 3   Id deuda                         2000 non-null   Int64         
 4   correo                           2000 non-null   object        
 5   Banco                            2000 non-null   object        
 6   Deuda Resuelve                   2000 non-null   float64       
 7   DBT                              2000 non-null   float64       
 8   tipo_cliente                     1977 non-null   object        
 9   Comisión Mensual                 2000 non-null   float64       
 10  Ahorro                           2000 non-null   int64         
 

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45714 entries, 0 to 45713
Data columns (total 59 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Referencia                 45714 non-null  object        
 1   Cedula                     45710 non-null  object        
 2   Nombre del cliente         45714 non-null  object        
 3   Id deuda                   45714 non-null  int64         
 4   correo                     45714 non-null  object        
 5   Deudas Activas             45714 non-null  int64         
 6   Banco                      45714 non-null  object        
 7   Número de Crédito          45714 non-null  object        
 8   Deuda Resuelve             45714 non-null  float64       
 9   DBT                        45609 non-null  float64       
 10  Meses de atraso            45714 non-null  int64         
 11  Dias de Atraso             45714 non-null  int64         
 12  Apar

In [23]:
# Asegurarnos de que estado_novacion sea texto y comparable
df["estado_novacion"] = df["estado_novacion"].astype(str)

# Filtrar referencias con estado NOVACION (sin importar mayúsculas)
refs_novacion = df.loc[
    df["estado_novacion"].str.upper() == "NOVACION",
    "Referencia"
].unique()

In [24]:
df_final_filtrado = df_final[
    ~df_final["Referencia"].isin(refs_novacion)
].copy()

In [25]:
df_final = df_final_filtrado.copy()

In [26]:
df_final

,Referencia,Cedula,Nombre del cliente,Id deuda,correo,Banco,Deuda Resuelve,DBT,tipo_cliente,Comisión Mensual,...,Ultima Liquidacion,dias_desde_ultima_liq,Priority_level,Apartado neto disponible,Depósitos para 30%,Apartados necesarios Pago Total,Ahorro / Deuda,Comisión total,ult_act,Pab
5,5829693,5829693,Rudi Andres lasso Giraldo,1862215,rudyandreslasso@hotmail.es,Falabella,1104258.0,49700682.0,bravo_co,249076.20,...,NaT,NaN,Prioridad 2,584505.0,3.618589,4.046986,0.366558,49277.51325,2025-10-23,159000000
6,5829693,5829693,Rudi Andres lasso Giraldo,1862219,rudyandreslasso@hotmail.es,Av Villas,4468142.0,49700682.0,bravo_co,249076.20,...,NaT,NaN,Prioridad 2,584505.0,4.914433,1.312854,0.533825,279147.17145,NaN,<NA>
7,5829693,5829693,Rudi Andres lasso Giraldo,1862218,rudyandreslasso@hotmail.es,Scotiabank Colpatria,4361500.0,49700682.0,bravo_co,249076.20,...,NaT,NaN,Prioridad 1,584505.0,2.672379,-0.656360,0.546878,516132.75000,2025-12-15,147000000
9,10932694,10932694,ARMANDO ILICK HUMANEZ USTA,1922148,armandousta@hotmail.com,Banco AV Villas,3383503.0,73687327.0,bravo_co,363251.44,...,NaT,NaN,Prioridad 1,592480.0,3.888485,1.885912,0.418396,249634.85134,NaN,<NA>
12,13715630,13715630,FREDY CASTELLANOS HERRENO,1840944,martilujosfredy@hotmail.com,Banco Finandina,3308800.0,126055900.0,bravo_co,462519.32,...,NaT,NaN,Prioridad 2,1212976.0,0.624522,6.184390,0.357846,225223.39840,2025-09-23,330880000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4920,3507660604,40032509,Maria Consuelo Puerto Hurtado,1935450,mcph35@hotmail.com,Itaú,3752700.0,98043730.0,bravo_co,479187.91,...,NaT,NaN,Prioridad 1,735671.0,4.246981,5.448418,0.198052,480064.14750,NaN,<NA>
4921,3507709067,43281300,Gloria Ines Gomez Restrepo,1956015,gloriagomezrestrepo66@gmail.com,Banco Popular,2328700.0,36413800.0,bravo_co,229163.06,...,NaT,NaN,Prioridad 1,312212.0,5.482110,4.741056,0.134340,116388.42600,NaN,<NA>
4924,3507978734,1090494112,VICTOR MANUEL PATINO HERNANDEZ,1991440,victorpatino1996@gmail.com,Scotiabank Colpatria,652520.0,8967483.0,bravo_co,65857.48,...,NaT,NaN,Prioridad 1,134894.0,3.418024,2.656493,0.196164,36107.19420,NaN,<NA>
4926,3508329735,1032480298,KAREN DAYANA RUEDA PENA,2016618,karueda03@hotmail.com,Falabella,1240800.0,81779500.0,bravo_co,0.00,...,NaT,NaN,Prioridad 2,1027618.0,2.034334,2.114591,0.000172,118124.16000,NaN,<NA>


In [27]:
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
import json
import os

# ======================================================
# 1) Detectar entorno y cargar MI_JSON
# ======================================================
def cargar_creds_dict():
    # --- Colab ---
    try:
        from google.colab import userdata
        mi_json = userdata.get("MI_JSON")
        if mi_json:
            print("Entorno detectado: Google Colab")
            return json.loads(mi_json)
    except Exception:
        pass

    # --- GitHub / local ---
    mi_json = os.environ.get("MI_JSON")
    if not mi_json:
        raise ValueError(
            "No encontré MI_JSON.\n"
            "- En Colab: define MI_JSON en Secrets\n"
            "- En GitHub/local: define MI_JSON como variable de entorno"
        )
    print("Entorno detectado: GitHub / local")
    return json.loads(mi_json)

creds_dict = cargar_creds_dict()

# ======================================================
# 2) Cliente gspread (service account)
# ======================================================
gc = gspread.service_account_from_dict(creds_dict)

# ======================================================
# 3) Abrir Google Sheet
# ======================================================
url = "https://docs.google.com/spreadsheets/d/1sMGQCCDiEzZI3f2w5BwFO8iWtxY5AEDJC2dypZ0NrCY/edit#gid=53625301"
sh = gc.open_by_url(url)

# ======================================================
# 4) Eliminar hoja "Priorizar" si existe
# ======================================================
try:
    worksheet_old = sh.worksheet("Priorizar")
    sh.del_worksheet(worksheet_old)
    print("Hoja 'Priorizar' eliminada para reemplazarla.")
except gspread.WorksheetNotFound:
    print("No existía hoja 'Priorizar', se creará nueva.")

# ======================================================
# 5) Crear nueva hoja
# ======================================================
worksheet = sh.add_worksheet(
    title="Priorizar",
    rows=str(len(df_final) + 5),
    cols=str(max(len(df_final.columns) + 5, 20))
)

# ======================================================
# 6) Subir DataFrame
# ======================================================
set_with_dataframe(worksheet, df_final, include_index=False)

print("Hoja 'Priorizar' creada y df_final cargado correctamente.")

Entorno detectado: Google Colab
Hoja 'Priorizar' eliminada para reemplazarla.
Hoja 'Priorizar' creada y df_final cargado correctamente.


In [28]:
import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe
import json
import os

# ======================================================
# 1) Detectar entorno y cargar MI_JSON
# ======================================================
def cargar_creds_dict():
    # --- Colab ---
    try:
        from google.colab import userdata
        mi_json = userdata.get("MI_JSON")
        if mi_json:
            print("Entorno detectado: Google Colab")
            return json.loads(mi_json)
    except Exception:
        pass

    # --- GitHub / local ---
    mi_json = os.environ.get("MI_JSON")
    if not mi_json:
        raise ValueError(
            "No encontré MI_JSON.\n"
            "- En Colab: define MI_JSON en Secrets\n"
            "- En GitHub/local: define MI_JSON como variable de entorno"
        )
    print("Entorno detectado: GitHub / local")
    return json.loads(mi_json)

creds_dict = cargar_creds_dict()

# ======================================================
# 2) Cliente gspread
# ======================================================
gc = gspread.service_account_from_dict(creds_dict)

# ======================================================
# 3) Abrir Google Sheet por URL
# ======================================================
url = "https://docs.google.com/spreadsheets/d/11xu99MBEZEQ8amR6S66opEXqLN8oMK01r7F5vb1L86M/edit#gid=1973783260"
gid = 1973783260

sh = gc.open_by_url(url)

# ======================================================
# 4) Seleccionar hoja por gid
# ======================================================
worksheet = sh.get_worksheet_by_id(gid)

# ======================================================
# 5) Convertir a DataFrame
# ======================================================
df_11xu99 = get_as_dataframe(worksheet, evaluate_formulas=True)

# Limpieza básica habitual
df_11xu99 = df_11xu99.dropna(how="all")

# ======================================================
# 6) Verificación
# ======================================================
df_11xu99.head()
df_11xu99.info()

Entorno detectado: Google Colab
<class 'pandas.core.frame.DataFrame'>
Index: 420 entries, 0 to 419
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   email               419 non-null    object
 1   employee_id         420 non-null    object
 2   name                420 non-null    object
 3   job_title           420 non-null    object
 4   leader              420 non-null    object
 5   status              420 non-null    object
 6   joined_resuelve_on  420 non-null    object
dtypes: object(7)
memory usage: 26.2+ KB


In [29]:
import pandas as pd
import unicodedata

# -----------------------------
# 1. Lista de negociadores
# -----------------------------
negociadores = [
    "Alba Yohana Moreno Martin",
    "Angie Lizeth Cubides Neira",
    "Angela Yara Mayorga",
    "Cindy Vanessa Cruz Rubio",
    "Maria Alejandra Bejarano Tumay",
    "Niyiret Julio Santos",
    "Rosa Yessenia Jimenez Lara",
    "Vivian Caterin Rodriguez Verano",
    "Francy Tatiana Sanchez Fontecha",

    "Dayana Isabel Ojito Ortiz",
    "Diego Alejandro Sanchez Fonseca",
    "Franchesca Julieth Pereira Candela",

    "Hector Elian Lacera Vega",
    "Jana Milena Lopez Buitrago",
    "Mauricio David Valencia Marquez",
    "Norbey Alejandro Duarte Ramirez",
    "Suleimy Tatiana Malaver Niño",
    "Sharon Natalia Hernandez Dominguez",
    "Wendy Vanessa Castillo Velasquez"

]

df_negociadores = pd.DataFrame({"name": negociadores})

# -----------------------------
# 2. Función para normalizar nombres
# -----------------------------
def normalizar_nombre(x):
    if pd.isna(x):
        return ""
    # quitar tildes
    x = unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('utf-8')
    # convertir en minúsculas
    x = x.lower()
    # quitar espacios dobles
    x = " ".join(x.split())
    # capitalizar cada palabra
    x = x.title()
    return x

# -----------------------------
# 3. Normalizar nombres en ambos DF
# -----------------------------
df_negociadores["name_clean"] = df_negociadores["name"].apply(normalizar_nombre)
df_11xu99["name_clean"] = df_11xu99["name"].astype(str).apply(normalizar_nombre)

# -----------------------------
# 4. Merge para obtener correo
# -----------------------------
df_negociadores_correo = df_negociadores.merge(
    df_11xu99[["name_clean", "email"]],
    on="name_clean",
    how="left"
)

# -----------------------------
# 5. Mostrar resultado final
# -----------------------------
print(df_negociadores_correo[["name", "email"]])

                                  name                              email
0            Alba Yohana Moreno Martin       yohana.moreno@gobravo.com.co
1           Angie Lizeth Cubides Neira       angie.cubides@gobravo.com.co
2                  Angela Yara Mayorga          ayara@resuelvetudeuda.com
3                  Angela Yara Mayorga         angela.yara@gobravo.com.co
4             Cindy Vanessa Cruz Rubio          cindy.cruz@gobravo.com.co
5       Maria Alejandra Bejarano Tumay     mariaa.bejarano@gobravo.com.co
6                 Niyiret Julio Santos      niyiret.santos@gobravo.com.co
7           Rosa Yessenia Jimenez Lara        rosa.jimenez@gobravo.com.co
8      Vivian Caterin Rodriguez Verano    vivian.rodriguez@gobravo.com.co
9          Nina Mayerly Gomez Guerrero          nina.gomez@gobravo.com.co
10     Francy Tatiana Sanchez Fontecha     tatiana.sanchez@gobravo.com.co
11           Dayana Isabel Ojito Ortiz        dayana.ojito@gobravo.com.co
12     Diego Alejandro Sanchez Fonseca

In [30]:
df_negociadores_correo

,name,name_clean,email
0,Alba Yohana Moreno Martin,Alba Yohana Moreno Martin,yohana.moreno@gobravo.com.co
1,Angie Lizeth Cubides Neira,Angie Lizeth Cubides Neira,angie.cubides@gobravo.com.co
2,Angela Yara Mayorga,Angela Yara Mayorga,ayara@resuelvetudeuda.com
3,Angela Yara Mayorga,Angela Yara Mayorga,angela.yara@gobravo.com.co
4,Cindy Vanessa Cruz Rubio,Cindy Vanessa Cruz Rubio,cindy.cruz@gobravo.com.co
5,Maria Alejandra Bejarano Tumay,Maria Alejandra Bejarano Tumay,mariaa.bejarano@gobravo.com.co
6,Niyiret Julio Santos,Niyiret Julio Santos,niyiret.santos@gobravo.com.co
7,Rosa Yessenia Jimenez Lara,Rosa Yessenia Jimenez Lara,rosa.jimenez@gobravo.com.co
8,Vivian Caterin Rodriguez Verano,Vivian Caterin Rodriguez Verano,vivian.rodriguez@gobravo.com.co
9,Nina Mayerly Gomez Guerrero,Nina Mayerly Gomez Guerrero,nina.gomez@gobravo.com.co


In [31]:
# 1. Marcar si el email pertenece a dominio gobravo
df_negociadores_correo["es_gobravo"] = df_negociadores_correo["email"].astype(str).str.contains("@gobravo.com.co")

# 2. Ordenar dejando primero los correos gobravo
df_negociadores_correo = df_negociadores_correo.sort_values(["name_clean", "es_gobravo"], ascending=[True, False])

# 3. Quitar duplicados quedándonos con el correo correcto
df_negociadores_correo = df_negociadores_correo.drop_duplicates(subset="name_clean", keep="first")

# 4. Limpiar la columna auxiliar
df_negociadores_correo = df_negociadores_correo.drop(columns=["es_gobravo"])

# 5. Ver resultado final
df_negociadores_correo

,name,name_clean,email
0,Alba Yohana Moreno Martin,Alba Yohana Moreno Martin,yohana.moreno@gobravo.com.co
3,Angela Yara Mayorga,Angela Yara Mayorga,angela.yara@gobravo.com.co
1,Angie Lizeth Cubides Neira,Angie Lizeth Cubides Neira,angie.cubides@gobravo.com.co
4,Cindy Vanessa Cruz Rubio,Cindy Vanessa Cruz Rubio,cindy.cruz@gobravo.com.co
11,Dayana Isabel Ojito Ortiz,Dayana Isabel Ojito Ortiz,dayana.ojito@gobravo.com.co
12,Diego Alejandro Sanchez Fonseca,Diego Alejandro Sanchez Fonseca,diego.sanchez@gobravo.com.co
13,Franchesca Julieth Pereira Candela,Franchesca Julieth Pereira Candela,franchesca.pereira@gobravo.com.co
10,Francy Tatiana Sanchez Fontecha,Francy Tatiana Sanchez Fontecha,tatiana.sanchez@gobravo.com.co
14,Hector Elian Lacera Vega,Hector Elian Lacera Vega,hector.lacera@gobravo.com.co
15,Jana Milena Lopez Buitrago,Jana Milena Lopez Buitrago,jana.lopez@gobravo.com.co


In [32]:
df_final.columns

Index(['Referencia', 'Cedula', 'Nombre del cliente', 'Id deuda', 'correo',
       'Banco', 'Deuda Resuelve', 'DBT', 'tipo_cliente', 'Comisión Mensual',
       'Ahorro', 'Tipo de cobro', 'Tipo de Liquidacion', 'CE', 'Fecha PL',
       'BANCOS_ESTANDAR', 'Descuento', 'P_cierre_Credito_norm', 'P_Cierre',
       'I_ESPERADO', 'Negociador', 'Tipo Elegible', 'P_Cierre_norm',
       'Días Atraso liquidacion', 'Estado PL', 'Ultima Liquidacion',
       'dias_desde_ultima_liq', 'Priority_level', 'Apartado neto disponible',
       'Depósitos para 30%', 'Apartados necesarios Pago Total',
       'Ahorro / Deuda', 'Comisión total', 'ult_act', 'Pab'],
      dtype='object')

In [33]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
import json
import os
import unicodedata
import pytz

# Zona horaria Colombia
tz_co = pytz.timezone("America/Bogota")

# ======================================================
# 0) Secrets portables (Colab / GitHub / local)
# ======================================================
def _get_secret(name: str):
    # Colab
    try:
        from google.colab import userdata
        v = userdata.get(name)
        if v is not None and str(v).strip() != "":
            print("Entorno detectado: Google Colab")
            return v
    except Exception:
        pass

    # GitHub / local
    v = os.environ.get(name)
    if v is not None and str(v).strip() != "":
        print("Entorno detectado: GitHub / local")
        return v

    return None

mi_json = _get_secret("MI_JSON")
if not mi_json:
    raise ValueError(
        "No encontré MI_JSON.\n"
        "- En Colab: define MI_JSON en Secrets\n"
        "- En GitHub/local: define MI_JSON como variable de entorno"
    )

gc = gspread.service_account_from_dict(json.loads(mi_json))

# ---------------------------------------
# 1. Normalizador de nombres
# ---------------------------------------
def normalizar_nombre(x):
    if pd.isna(x):
        return ""
    x = unicodedata.normalize("NFKD", str(x)).encode("ascii", "ignore").decode()
    x = " ".join(x.lower().split())
    return x.title()

# ---------------------------------------
# 3. Abrir Sheet de historial "Asignar"
# ---------------------------------------
sheet_url = "https://docs.google.com/spreadsheets/d/1sMGQCCDiEzZI3f2w5BwFO8iWtxY5AEDJC2dypZ0NrCY/edit"
sh = gc.open_by_url(sheet_url)
ws_asignar = sh.worksheet("Asignar")

df_hist = get_as_dataframe(ws_asignar, evaluate_formulas=True).dropna(how="all")

# 🔹 Historial ahora se maneja por Id deuda (NO por Referencia)
if "Id deuda" in df_hist.columns:
    df_hist["Id deuda"] = pd.to_numeric(df_hist["Id deuda"], errors="coerce").astype("Int64")
    ids_previos = set(df_hist["Id deuda"].dropna().tolist())
else:
    ids_previos = set()

# ---------------------------------------
# 4. Preparar df_final (trabajamos por Referencia pero bloqueamos por Id deuda)
# ---------------------------------------
df_final = df_final.copy()

df_final["Referencia"] = pd.to_numeric(df_final["Referencia"], errors="coerce").astype("Int64")
df_final["Id deuda"]   = pd.to_numeric(df_final["Id deuda"], errors="coerce").astype("Int64")

# 🔴 EXCLUIR SIEMPRE LAS REFERENCIAS DE WILLIAM (NO SE ASIGNAN A NADIE)
df_final = df_final[df_final["Negociador"].astype(str) != "William Santiago Abril Esguerra"].copy()

df_final["neg_clean"] = df_final["Negociador"].astype(str).apply(normalizar_nombre)
df_final["ult_act"]   = pd.to_datetime(df_final["ult_act"], errors="coerce")

hoy    = datetime.now().date()
limite = hoy - timedelta(days=60)

df_candidatos = df_final[
    (
        df_final["ult_act"].isna() |
        (df_final["ult_act"].dt.date < limite)
    )
    &
    (~df_final["Id deuda"].isin(ids_previos))
]

# ---------------------------------------
# 5. Lista principal de negociadores válidos
# ---------------------------------------
df_negociadores_correo["name_clean"] = df_negociadores_correo["name"].apply(normalizar_nombre)
lista_principal = df_negociadores_correo["name_clean"].unique().tolist()

# ---------------------------------------
# 6. Fallback
# ---------------------------------------
fallback_pool = df_candidatos[~df_candidatos["neg_clean"].isin(lista_principal)].copy()

# ---------------------------------------
# 7. Asignación diaria (3 REFERENCIAS por negociador principal)
# ---------------------------------------
asignaciones = []
referencias_asignadas_run = set()

for neg in lista_principal:
    pool = df_candidatos[
        (df_candidatos["neg_clean"] == neg) &
        (~df_candidatos["Referencia"].isin(referencias_asignadas_run))
    ].copy()

    pool = pool.sort_values(by=["Comisión total", "Ahorro"], ascending=False)
    pool = pool.drop_duplicates(subset="Referencia", keep="first")

    seleccion = pool.head(3)
    faltan = 3 - len(seleccion)

    if faltan > 0:
        pool_fb = fallback_pool[
            ~fallback_pool["Referencia"].isin(referencias_asignadas_run)
        ].copy()
        pool_fb = pool_fb.sort_values(by=["Comisión total", "Ahorro"], ascending=False)
        pool_fb = pool_fb.drop_duplicates(subset="Referencia", keep="first")

        fb_take = pool_fb.head(faltan)
        seleccion = pd.concat([seleccion, fb_take])

        fallback_pool = fallback_pool[~fallback_pool["Referencia"].isin(fb_take["Referencia"])]

    if len(seleccion) < 3:
        print(f"⚠️ {neg} no tiene suficientes referencias disponibles. No se asigna hoy.")
        continue

    referencias_asignadas_run.update(seleccion["Referencia"].tolist())

    seleccion = seleccion.copy()
    seleccion["Negociador asignado"] = neg

    email = df_negociadores_correo.loc[df_negociadores_correo["name_clean"] == neg, "email"].iloc[0]
    seleccion["Email asignado"] = email
    seleccion["Fecha asignación"] = datetime.now(tz_co).strftime("%Y-%m-%d %H:%M:%S")
    seleccion["Negociador original"] = seleccion["Negociador"]

    asignaciones.append(seleccion)

if len(asignaciones) == 0:
    print("⚠️ No se generaron asignaciones hoy.")
    df_asignar_hoy = pd.DataFrame(columns=df_final.columns.tolist() + [
        "Negociador asignado", "Email asignado", "Fecha asignación", "Negociador original"
    ])
else:
    df_asignar_hoy = pd.concat(asignaciones, ignore_index=True)

# ---------------------------------------
# 8. Columnas finales del Sheet
# ---------------------------------------
columnas_finales_sheet = [
    "Negociador asignado",
    "Email asignado",
    "Negociador original",
    "Fecha asignación",
    "ult_act",
    "Referencia",
    "Id deuda",
    "Depósitos para 30%",
    "Comisión total",
    "Descuento"
]

df_asignar_hoy_sheet = df_asignar_hoy[columnas_finales_sheet]

df_hist_sheet = (
    df_hist[columnas_finales_sheet]
    if len(df_hist) > 0
    else pd.DataFrame(columns=columnas_finales_sheet)
)

df_total_sheet = pd.concat([df_asignar_hoy_sheet, df_hist_sheet], ignore_index=True)

# ---------------------------------------
# 9. ESCRIBIR ASIGNACIONES
# ---------------------------------------
ws_asignar.clear()
set_with_dataframe(ws_asignar, df_total_sheet)
print("✔ Asignaciones agregadas en la hoja 'Asignar'")
print("➡ Filas nuevas agregadas:", len(df_asignar_hoy_sheet))

# ---------------------------------------
# 10. AGREGAR COLUMNA ¿Se actualizó?
# ---------------------------------------
df_12098["bank_reference"] = pd.to_numeric(df_12098["bank_reference"], errors="coerce").astype("Int64")

def limpiar_fecha(x):
    try:
        return pd.to_datetime(x).date()
    except Exception:
        return pd.NaT

df_12098["inserted_at"] = df_12098["inserted_at"].apply(limpiar_fecha)
df_12098_valid = df_12098.dropna(subset=["inserted_at", "bank_reference"])

ult_act_dict = df_12098_valid.groupby("bank_reference")["inserted_at"].max().to_dict()

df_total_sheet["Fila vieja?"] = 0
df_total_sheet.loc[len(df_asignar_hoy_sheet):, "Fila vieja?"] = 1

def verificar_actualizacion(row):
    if row["Fila vieja?"] == 0:
        return ""

    ref = row["Referencia"]
    asign = row["Fecha asignación"]

    try:
        asign = pd.to_datetime(asign).date()
    except Exception:
        return "Fecha asignación inválida"

    if pd.isna(ref):
        return "Sin información"

    if ref not in ult_act_dict:
        return "Sin información"

    inserted = ult_act_dict[ref]
    return "Sí se actualizó" if inserted > asign else "No se actualizó"

df_total_sheet["¿Se actualizó?"] = df_total_sheet.apply(verificar_actualizacion, axis=1)
df_total_sheet = df_total_sheet.drop(columns=["Fila vieja?"])

# ---------------------------------------
# 11. REESCRIBIR HOJA CON COLUMNA ¿Se actualizó?
# ---------------------------------------
ws_asignar.clear()
set_with_dataframe(ws_asignar, df_total_sheet)
print("✔ Columna '¿Se actualizó?' agregada correctamente.")

Entorno detectado: Google Colab
✔ Asignaciones agregadas en la hoja 'Asignar'
➡ Filas nuevas agregadas: 60
✔ Columna '¿Se actualizó?' agregada correctamente.


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import json
import os
import pytz
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe

# ======================================================
# 0) Secrets portables (Colab / GitHub / local)
# ======================================================
def _get_secret(name: str):
    try:
        from google.colab import userdata
        v = userdata.get(name)
        if v is not None and str(v).strip() != "":
            return v
    except Exception:
        pass
    v = os.environ.get(name)
    if v is not None and str(v).strip() != "":
        return v
    return None

mi_json = _get_secret("MI_JSON")
if not mi_json:
    raise ValueError(
        "No encontré MI_JSON.\n"
        "- En Colab: define MI_JSON en Secrets\n"
        "- En GitHub/local: define MI_JSON como variable de entorno"
    )

# Zona horaria Colombia (portable)
tz_co = pytz.timezone("America/Bogota")

# Conectar a Sheets (portable)
gc = gspread.service_account_from_dict(json.loads(mi_json))

sheet_url = "https://docs.google.com/spreadsheets/d/1sMGQCCDiEzZI3f2w5BwFO8iWtxY5AEDJC2dypZ0NrCY/edit"
sh = gc.open_by_url(sheet_url)

# ---------------------------------------
# NUEVO BLOQUE: Resumen para Bot por negociador (solo HOY)
# ---------------------------------------

# Volvemos a leer la hoja 'Asignar' ya actualizada
ws_asignar = sh.worksheet("Asignar")
df_asignar = get_as_dataframe(ws_asignar, evaluate_formulas=True).dropna(how="all")

# Aseguramos que existe 'Fecha asignación'
if "Fecha asignación" not in df_asignar.columns:
    raise ValueError("La hoja 'Asignar' no tiene la columna 'Fecha asignación'.")

# Parsear Fecha asignación a datetime
df_asignar["Fecha asignación"] = pd.to_datetime(df_asignar["Fecha asignación"], errors="coerce")

# Día de hoy (zona horaria Colombia)
hoy_co = datetime.now(tz_co).date()

# Filtrar solo filas de hoy
df_hoy = df_asignar[df_asignar["Fecha asignación"].dt.date == hoy_co].copy()

# Si no hay filas para hoy → DF vacío con estructura final
if df_hoy.empty:
    print("⚠️ No hay asignaciones para el día de hoy en la hoja 'Asignar'.")
    df_bot = pd.DataFrame(columns=[
        "Negociador asignado",
        "Email asignado",
        "Fecha asignación",
        "Mensaje"
    ])
else:
    columnas_necesarias = [
        "Negociador asignado",
        "Email asignado",
        "Fecha asignación",
        "Referencia",
        "Id deuda",
        "Depósitos para 30%",
        "Comisión total",
        "Descuento"
    ]
    for col in columnas_necesarias:
        if col not in df_hoy.columns:
            df_hoy[col] = np.nan

    # Tipos
    df_hoy["Referencia"] = pd.to_numeric(df_hoy["Referencia"], errors="coerce").astype("Int64")
    df_hoy["Id deuda"]   = pd.to_numeric(df_hoy["Id deuda"], errors="coerce").astype("Int64")
    for col_num in ["Depósitos para 30%", "Comisión total", "Descuento"]:
        df_hoy[col_num] = pd.to_numeric(df_hoy[col_num], errors="coerce")

    # Construir una línea de detalle por cada fila
    def construir_linea(row):
        ref   = row["Referencia"]
        deuda = row["Id deuda"]
        dep   = row["Depósitos para 30%"]
        com   = row["Comisión total"]
        desc  = row["Descuento"]

        ref_txt   = f"{ref}" if pd.notna(ref) else "N/A"
        deuda_txt = f"{deuda}" if pd.notna(deuda) else "N/A"
        dep_txt   = f"{dep:,.0f}" if pd.notna(dep) else "N/A"
        com_txt   = f"{com:,.0f}" if pd.notna(com) else "N/A"
        desc_txt  = f"{desc:.2%}" if pd.notna(desc) else "N/A"

        return (
            f"Referencia: {ref_txt} | "
            f"Id deuda: {deuda_txt} | "
            f"Depósitos 30%: {dep_txt} | "
            f"Comisión total: {com_txt} | "
            f"Descuento: {desc_txt}"
        )

    df_hoy["Detalle"] = df_hoy.apply(construir_linea, axis=1)

    # Agrupar y unir líneas
    df_bot = (
        df_hoy
        .groupby(["Negociador asignado", "Email asignado", "Fecha asignación"], as_index=False)
        .agg({"Detalle": lambda x: "\n".join(x)})
        .rename(columns={"Detalle": "Mensaje"})
    )

    df_bot = df_bot[["Negociador asignado", "Email asignado", "Fecha asignación", "Mensaje"]]

# ---------------------------------------
# Escribir SIEMPRE la hoja 'Asignar Bot'
# ---------------------------------------
try:
    ws_bot = sh.worksheet("Asignar Bot")
except gspread.WorksheetNotFound:
    ws_bot = sh.add_worksheet(title="Asignar Bot", rows="100", cols="10")

ws_bot.clear()
set_with_dataframe(ws_bot, df_bot, include_index=False)

print("✔ Hoja 'Asignar Bot' actualizada con los mensajes por negociador del día de hoy.")

✔ Hoja 'Asignar Bot' actualizada con los mensajes por negociador del día de hoy.


#Seguimiento

In [77]:
import json
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials

# =====================================
# 1. Cargar MI_JSON (Colab / GitHub)
# =====================================
try:
    from google.colab import userdata
    mi_json = userdata.get("MI_JSON")
except ImportError:
    import os
    mi_json = os.environ.get("MI_JSON")

if mi_json is None:
    raise ValueError("MI_JSON no encontrado")

creds_dict = json.loads(mi_json)

scopes = ["https://www.googleapis.com/auth/spreadsheets.readonly"]
creds = Credentials.from_service_account_info(creds_dict, scopes=scopes)

gc = gspread.authorize(creds)

# =====================================
# 2. Abrir Sheet
# =====================================
SPREADSHEET_ID = "1O8OHuVhgwhLw8XYEBf1uBzLYrxQ45rPiZecHOnAa1Go"
GID = 1739799528

sh = gc.open_by_key(SPREADSHEET_ID)
ws = sh.get_worksheet_by_id(GID)
df_act = pd.DataFrame(ws.get_all_records())




In [78]:
df_act

,expiration,observations,end,debt_id,payment_to_bank,inserted_at,bank_reference
0,,Stefy 8086 // 7:22 // Se valida en chat de tuy...,stefany.rojas@gobravo.com.co,1508614,"(551000000,COP)",2025-01-02 12:23:03,51911010
1,,7:22 a. m. 02/01/2025 VivianR//8522// Crédito ...,vivian.rodriguez@gobravo.com.co,1186799,"(500000000,COP)",2025-01-02 12:48:27,3108379218
2,,YesseniaJ|8045|02-01-2025|9:07am|llamo a tt pa...,rosa.jimenez@gobravo.com.co,1305562,"(54700000,COP)",2025-01-02 14:08:00,3103940951
3,,Actividad para Nicolle Stefy 8086 // 10:52 // ...,stefany.rojas@gobravo.com.co,873948,"(2600000000,COP)",2025-01-02 15:58:43,3158318257
4,,7:22 a. m. 02/01/2025 VivianR//8522// Crédito ...,vivian.rodriguez@gobravo.com.co,1450752,"(573800000,COP)",2025-01-02 12:44:39,3022621202
...,...,...,...,...,...,...,...
280137,,3:49 p. m. 15/12/2025 /Maleja/8489/ Se valida ...,mariaa.bejarano@gobravo.com.co,1814383,"(61088400,COP)",2025-12-15 20:50:07,3208533559
280138,,3:26 p. m. 15/12/2025||Diego S ||se valida por...,diego.sanchez@gobravo.com.co,1909973,"(100000000,COP)",2025-12-15 20:30:47,1018477273
280139,,4:16 p. m. Dayana O. ext 8041 // Se recibe des...,dayana.ojito@gobravo.com.co,1813310,"(150000000,COP)",2025-12-15 21:17:18,3166994965
280140,,"4:06 p.m| Niyiret8068 || Re recibe dt.$53,777,...",niyiret.santos@gobravo.com.co,1709766,"(2688863500,COP)",2025-12-15 21:07:19,3105906960


In [79]:
import json
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials

# =====================================
# 1. Cargar MI_JSON (Colab / GitHub)
# =====================================
try:
    from google.colab import userdata
    mi_json = userdata.get("MI_JSON")
except ImportError:
    import os
    mi_json = os.environ.get("MI_JSON")

if mi_json is None:
    raise ValueError("MI_JSON no encontrado")

creds_dict = json.loads(mi_json)

scopes = ["https://www.googleapis.com/auth/spreadsheets.readonly"]
creds = Credentials.from_service_account_info(creds_dict, scopes=scopes)

gc = gspread.authorize(creds)

# =====================================
# 2. Abrir Sheet
# =====================================
SPREADSHEET_ID = "1sMGQCCDiEzZI3f2w5BwFO8iWtxY5AEDJC2dypZ0NrCY"
GID = 1129967215

sh = gc.open_by_key(SPREADSHEET_ID)
ws = sh.get_worksheet_by_id(GID)
df_seg = pd.DataFrame(ws.get_all_records())

In [80]:
df_seg

,Negociador asignado,Email asignado,Negociador original,Fecha asignación,ult_act,Referencia,Id deuda,Depósitos para 30%,Comisión total,Descuento
0,Alba Yohana Moreno Martin,yohana.moreno@gobravo.com.co,Alba Yohana Moreno Martin,2025-12-15 8:36:56,,14607860,1826984,-5314823281,2166273144,57
1,Alba Yohana Moreno Martin,yohana.moreno@gobravo.com.co,Alba Yohana Moreno Martin,2025-12-15 8:36:56,,79608961,1782541,-1030314596,180414234,45
2,Alba Yohana Moreno Martin,yohana.moreno@gobravo.com.co,Alba Yohana Moreno Martin,2025-12-15 8:36:56,,3103147559,1941825,511363302,7157111629,1
3,Angela Yara Mayorga,angela.yara@gobravo.com.co,Angela Yara Mayorga,2025-12-15 8:36:56,,3232353979,1816967,5707772069,141920709,475
4,Angela Yara Mayorga,angela.yara@gobravo.com.co,Angela Yara Mayorga,2025-12-15 8:36:56,2025-07-28 0:00:00,3007905364,1790991,4853006449,1178288496,4852
...,...,...,...,...,...,...,...,...,...,...
175,Vivian Caterin Rodriguez Verano,vivian.rodriguez@gobravo.com.co,Vivian Caterin Rodriguez Verano,2025-12-11 12:25:22,2025-09-19 0:00:00,94315912,1745159,1907272358,2044856968,62
176,Vivian Caterin Rodriguez Verano,vivian.rodriguez@gobravo.com.co,Vivian Caterin Rodriguez Verano,2025-12-11 12:25:22,,3163937546,1950291,2677885346,1734266016,48
177,Wendy Vanessa Castillo Velasquez,wendy.castillo@gobravo.com.co,Wendy Vanessa Castillo Velasquez,2025-12-11 12:25:22,,3117293523,1921130,5809857922,29640996,6
178,Wendy Vanessa Castillo Velasquez,wendy.castillo@gobravo.com.co,Wendy Vanessa Castillo Velasquez,2025-12-11 12:25:22,,3168693335,1831247,5422921029,1003761241,6987


In [81]:
df_seg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Negociador asignado  180 non-null    object
 1   Email asignado       180 non-null    object
 2   Negociador original  180 non-null    object
 3   Fecha asignación     180 non-null    object
 4   ult_act              180 non-null    object
 5   Referencia           180 non-null    int64 
 6   Id deuda             180 non-null    int64 
 7   Depósitos para 30%   180 non-null    int64 
 8   Comisión total       180 non-null    int64 
 9   Descuento            180 non-null    int64 
dtypes: int64(5), object(5)
memory usage: 14.2+ KB


In [82]:
df_act.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280142 entries, 0 to 280141
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   expiration       280142 non-null  object
 1   observations     280142 non-null  object
 2   end              280142 non-null  object
 3   debt_id          280142 non-null  int64 
 4   payment_to_bank  280142 non-null  object
 5   inserted_at      280142 non-null  object
 6   bank_reference   280142 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 15.0+ MB


In [83]:
import pandas as pd

# Copia de seguridad
df_seg = df_seg.copy()

# Normalizar nombres de columnas (por si vienen con espacios raros)
df_seg.columns = (
    df_seg.columns
    .str.strip()
)

# Fecha asignación a datetime
df_seg["Fecha asignación"] = pd.to_datetime(
    df_seg["Fecha asignación"],
    errors="coerce"
)

# Referencia e Id deuda como texto (clave)
df_seg["Referencia"] = df_seg["Referencia"].astype(str)
df_seg["Id deuda"] = df_seg["Id deuda"].astype(str)

In [84]:
# Quedarse únicamente con las columnas necesarias
cols = [
    "Negociador asignado",
    "Fecha asignación",
    "Referencia",
    "Id deuda"
]

df_seg = df_seg[cols].copy()

In [85]:
df_seg

,Negociador asignado,Fecha asignación,Referencia,Id deuda
0,Alba Yohana Moreno Martin,2025-12-15 08:36:56,14607860,1826984
1,Alba Yohana Moreno Martin,2025-12-15 08:36:56,79608961,1782541
2,Alba Yohana Moreno Martin,2025-12-15 08:36:56,3103147559,1941825
3,Angela Yara Mayorga,2025-12-15 08:36:56,3232353979,1816967
4,Angela Yara Mayorga,2025-12-15 08:36:56,3007905364,1790991
...,...,...,...,...
175,Vivian Caterin Rodriguez Verano,2025-12-11 12:25:22,94315912,1745159
176,Vivian Caterin Rodriguez Verano,2025-12-11 12:25:22,3163937546,1950291
177,Wendy Vanessa Castillo Velasquez,2025-12-11 12:25:22,3117293523,1921130
178,Wendy Vanessa Castillo Velasquez,2025-12-11 12:25:22,3168693335,1831247


In [86]:
df_act.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280142 entries, 0 to 280141
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   expiration       280142 non-null  object
 1   observations     280142 non-null  object
 2   end              280142 non-null  object
 3   debt_id          280142 non-null  int64 
 4   payment_to_bank  280142 non-null  object
 5   inserted_at      280142 non-null  object
 6   bank_reference   280142 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 15.0+ MB


In [87]:
import pandas as pd

# Copia por seguridad
df_act = df_act.copy()

# Convertir a object (string)
df_act["bank_reference"] = df_act["bank_reference"].astype(str)
df_act["debt_id"] = df_act["debt_id"].astype(str)

# Convertir inserted_at a datetime
df_act["inserted_at"] = pd.to_datetime(
    df_act["inserted_at"],
    errors="coerce"
)

# Renombrar columnas
df_act = df_act.rename(columns={
    "bank_reference": "Referencia",
    "debt_id": "Id deuda",
    "inserted_at": "Fecha act"
})

In [88]:
df_act

,expiration,observations,end,Id deuda,payment_to_bank,Fecha act,Referencia
0,,Stefy 8086 // 7:22 // Se valida en chat de tuy...,stefany.rojas@gobravo.com.co,1508614,"(551000000,COP)",2025-01-02 12:23:03,51911010
1,,7:22 a. m. 02/01/2025 VivianR//8522// Crédito ...,vivian.rodriguez@gobravo.com.co,1186799,"(500000000,COP)",2025-01-02 12:48:27,3108379218
2,,YesseniaJ|8045|02-01-2025|9:07am|llamo a tt pa...,rosa.jimenez@gobravo.com.co,1305562,"(54700000,COP)",2025-01-02 14:08:00,3103940951
3,,Actividad para Nicolle Stefy 8086 // 10:52 // ...,stefany.rojas@gobravo.com.co,873948,"(2600000000,COP)",2025-01-02 15:58:43,3158318257
4,,7:22 a. m. 02/01/2025 VivianR//8522// Crédito ...,vivian.rodriguez@gobravo.com.co,1450752,"(573800000,COP)",2025-01-02 12:44:39,3022621202
...,...,...,...,...,...,...,...
280137,,3:49 p. m. 15/12/2025 /Maleja/8489/ Se valida ...,mariaa.bejarano@gobravo.com.co,1814383,"(61088400,COP)",2025-12-15 20:50:07,3208533559
280138,,3:26 p. m. 15/12/2025||Diego S ||se valida por...,diego.sanchez@gobravo.com.co,1909973,"(100000000,COP)",2025-12-15 20:30:47,1018477273
280139,,4:16 p. m. Dayana O. ext 8041 // Se recibe des...,dayana.ojito@gobravo.com.co,1813310,"(150000000,COP)",2025-12-15 21:17:18,3166994965
280140,,"4:06 p.m| Niyiret8068 || Re recibe dt.$53,777,...",niyiret.santos@gobravo.com.co,1709766,"(2688863500,COP)",2025-12-15 21:07:19,3105906960


In [89]:
import pandas as pd
import numpy as np

# -----------------------------
# 0) Copias y tipos
# -----------------------------
df_seg = df_seg.copy()
df_act = df_act.copy()

# Llaves como texto
for c in ["Id deuda", "Referencia"]:
    if c in df_seg.columns:
        df_seg[c] = df_seg[c].astype(str)
    if c in df_act.columns:
        df_act[c] = df_act[c].astype(str)

# Fechas
df_seg["Fecha asignación"] = pd.to_datetime(df_seg["Fecha asignación"], errors="coerce")

# df_act: asegurar Fecha act
if "Fecha act" not in df_act.columns:
    if "Fecha asignación" in df_act.columns:
        df_act["Fecha act"] = pd.to_datetime(df_act["Fecha asignación"], errors="coerce")
    elif "inserted_at" in df_act.columns:
        df_act["Fecha act"] = pd.to_datetime(df_act["inserted_at"], errors="coerce")
    else:
        raise ValueError("df_act no tiene 'Fecha act' ni una alternativa ('Fecha asignación' o 'inserted_at').")

df_act["Fecha act"] = pd.to_datetime(df_act["Fecha act"], errors="coerce")

# Comparación por día (sin hora)
df_seg["_fa_dia"] = df_seg["Fecha asignación"].dt.floor("D")
df_act["_fact_dia"] = df_act["Fecha act"].dt.floor("D")

# -----------------------------
# 1) MATCH 1: por Id deuda (Esa deuda)
# -----------------------------
tmp_deuda = df_seg[["Id deuda", "Fecha asignación", "_fa_dia"]].merge(
    df_act[["Id deuda", "Fecha act", "_fact_dia", "observations"]],
    on="Id deuda",
    how="left"
)

tmp_deuda_val = tmp_deuda[
    tmp_deuda["Fecha act"].notna()
    & tmp_deuda["_fa_dia"].notna()
    & (tmp_deuda["_fact_dia"] >= tmp_deuda["_fa_dia"])
].copy()

tmp_deuda_val = tmp_deuda_val.sort_values(["Id deuda", "Fecha asignación", "Fecha act"])
best_deuda = tmp_deuda_val.drop_duplicates(subset=["Id deuda", "Fecha asignación"], keep="first")

df_seg = df_seg.merge(
    best_deuda[["Id deuda", "Fecha asignación", "Fecha act", "observations"]],
    on=["Id deuda", "Fecha asignación"],
    how="left",
    suffixes=("", "")
)

# Crear columna como texto (dtype object) y luego asignar
df_seg["que?"] = pd.NA
mask_esa = df_seg["Fecha act"].notna() | df_seg["observations"].notna()
df_seg.loc[mask_esa, "que?"] = "Esa deuda"

# -----------------------------
# 2) MATCH 2: fallback por Referencia (Otra deuda)
#     Solo para filas que siguen sin gestión
# -----------------------------
mask_falta = df_seg["Fecha act"].isna() & df_seg["observations"].isna()

tmp_ref = df_seg.loc[mask_falta, ["Referencia", "Fecha asignación", "_fa_dia"]].merge(
    df_act[["Referencia", "Fecha act", "_fact_dia", "observations"]],
    on="Referencia",
    how="left"
)

tmp_ref_val = tmp_ref[
    tmp_ref["Fecha act"].notna()
    & tmp_ref["_fa_dia"].notna()
    & (tmp_ref["_fact_dia"] >= tmp_ref["_fa_dia"])
].copy()

# Elegir la Fecha act más cercana (mínima) para esa Referencia + Fecha asignación
tmp_ref_val = tmp_ref_val.sort_values(["Referencia", "Fecha asignación", "Fecha act"])
best_ref = tmp_ref_val.drop_duplicates(subset=["Referencia", "Fecha asignación"], keep="first")

# Volver a pegarlo en df_seg SOLO para las filas faltantes
df_seg = df_seg.merge(
    best_ref[["Referencia", "Fecha asignación", "Fecha act", "observations"]],
    on=["Referencia", "Fecha asignación"],
    how="left",
    suffixes=("", "_ref")
)

# Si se llenó por referencia, reemplazamos los nulos originales
df_seg["Fecha act"] = df_seg["Fecha act"].combine_first(df_seg["Fecha act_ref"])
df_seg["observations"] = df_seg["observations"].combine_first(df_seg["observations_ref"])

# Marcar "que?" como Otra deuda donde se completó por referencia
df_seg.loc[
    mask_falta & (df_seg["Fecha act_ref"].notna() | df_seg["observations_ref"].notna()),
    "que?"
] = "Otra deuda"

# Limpiar auxiliares
df_seg = df_seg.drop(columns=["_fa_dia", "Fecha act_ref", "observations_ref"], errors="ignore")
df_act = df_act.drop(columns=["_fact_dia"], errors="ignore")

In [90]:
df_seg

,Negociador asignado,Fecha asignación,Referencia,Id deuda,Fecha act,observations,que?
0,Alba Yohana Moreno Martin,2025-12-15 08:36:56,14607860,1826984,NaT,NaN,<NA>
1,Alba Yohana Moreno Martin,2025-12-15 08:36:56,79608961,1782541,NaT,NaN,<NA>
2,Alba Yohana Moreno Martin,2025-12-15 08:36:56,3103147559,1941825,NaT,NaN,<NA>
3,Angela Yara Mayorga,2025-12-15 08:36:56,3232353979,1816967,2025-12-15 18:02:38,Ange//Ext8433//1:00pm//Se hace validación por ...,Esa deuda
4,Angela Yara Mayorga,2025-12-15 08:36:56,3007905364,1790991,NaT,NaN,<NA>
...,...,...,...,...,...,...,...
175,Vivian Caterin Rodriguez Verano,2025-12-11 12:25:22,94315912,1745159,2025-12-12 15:28:32,10:27 a. m. 12/12/2025 VivianR// tt indica en ...,Esa deuda
176,Vivian Caterin Rodriguez Verano,2025-12-11 12:25:22,3163937546,1950291,NaT,NaN,<NA>
177,Wendy Vanessa Castillo Velasquez,2025-12-11 12:25:22,3117293523,1921130,2025-12-12 00:39:36,Actualizado desde alianzas,Otra deuda
178,Wendy Vanessa Castillo Velasquez,2025-12-11 12:25:22,3168693335,1831247,2025-12-15 17:39:01,Vanessa Castillo/12: 38 pm//-15-12- 2025/ lle...,Otra deuda


In [91]:
import numpy as np

df_seg = df_seg.copy()

df_seg["se gestionó"] = np.where(
    df_seg["Fecha act"].isna() & df_seg["observations"].isna(),
    "No",
    "Sí"
)

In [92]:
df_seg

,Negociador asignado,Fecha asignación,Referencia,Id deuda,Fecha act,observations,que?,se gestionó
0,Alba Yohana Moreno Martin,2025-12-15 08:36:56,14607860,1826984,NaT,NaN,<NA>,No
1,Alba Yohana Moreno Martin,2025-12-15 08:36:56,79608961,1782541,NaT,NaN,<NA>,No
2,Alba Yohana Moreno Martin,2025-12-15 08:36:56,3103147559,1941825,NaT,NaN,<NA>,No
3,Angela Yara Mayorga,2025-12-15 08:36:56,3232353979,1816967,2025-12-15 18:02:38,Ange//Ext8433//1:00pm//Se hace validación por ...,Esa deuda,Sí
4,Angela Yara Mayorga,2025-12-15 08:36:56,3007905364,1790991,NaT,NaN,<NA>,No
...,...,...,...,...,...,...,...,...
175,Vivian Caterin Rodriguez Verano,2025-12-11 12:25:22,94315912,1745159,2025-12-12 15:28:32,10:27 a. m. 12/12/2025 VivianR// tt indica en ...,Esa deuda,Sí
176,Vivian Caterin Rodriguez Verano,2025-12-11 12:25:22,3163937546,1950291,NaT,NaN,<NA>,No
177,Wendy Vanessa Castillo Velasquez,2025-12-11 12:25:22,3117293523,1921130,2025-12-12 00:39:36,Actualizado desde alianzas,Otra deuda,Sí
178,Wendy Vanessa Castillo Velasquez,2025-12-11 12:25:22,3168693335,1831247,2025-12-15 17:39:01,Vanessa Castillo/12: 38 pm//-15-12- 2025/ lle...,Otra deuda,Sí


In [93]:
import json
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials
from gspread_dataframe import set_with_dataframe

# =========================================================
# 1. Cargar credenciales MI_JSON (Colab / GitHub)
# =========================================================
try:
    # Google Colab
    from google.colab import userdata
    mi_json = userdata.get("MI_JSON")
except ImportError:
    # GitHub / local
    import os
    mi_json = os.environ.get("MI_JSON")

if mi_json is None:
    raise ValueError("No se encontró MI_JSON en el entorno.")

creds_dict = json.loads(mi_json)

scopes = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

creds = Credentials.from_service_account_info(creds_dict, scopes=scopes)
gc = gspread.authorize(creds)

# =========================================================
# 2. Abrir el Google Sheet destino
# =========================================================
SPREADSHEET_ID = "1sMGQCCDiEzZI3f2w5BwFO8iWtxY5AEDJC2dypZ0NrCY"
SHEET_NAME = "Gestion"

sh = gc.open_by_key(SPREADSHEET_ID)

# =========================================================
# 3. Crear hoja "Gestion" (o limpiarla si ya existe)
# =========================================================
try:
    ws = sh.worksheet(SHEET_NAME)
    ws.clear()
except gspread.exceptions.WorksheetNotFound:
    ws = sh.add_worksheet(
        title=SHEET_NAME,
        rows=str(len(df_seg) + 5),
        cols=str(len(df_seg.columns) + 5)
    )

# =========================================================
# 4. Escribir df_seg en la hoja
# =========================================================
set_with_dataframe(
    ws,
    df_seg,
    include_index=False,
    include_column_header=True,
    resize=True
)

print("✅ df_seg escrito correctamente en la hoja 'Gestion'")

✅ df_seg escrito correctamente en la hoja 'Gestion'
